# **Module 4: Spatial Dependence and Regression**
## **Prerequisites**
### Data
For this workshop, data are created and saved to the directory `./data-module-4/`.
- `mnp.shp` -  a pseudo dataset representing hypothetical pest stress for selected Minnesota counties.
### Software
To execute the code you will need a `conda` environment for Python with the packages imported below.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pysal.lib import weights
from spreg import ML_Error_Regimes, ML_Lag, ML_Error, OLS
from splot.libpysal import plot_spatial_weights
import esda 
from splot.esda import plot_moran

### Read and visualize input data

In [ ]:
mnp = gpd.read_file("./data-module-4/mnp.shp")
print ("Coordinate reference system is {}".format(mnp.crs))
print ("Number of records is {}".format(len(mnp)))
mnp.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
mnp.plot(ax=ax, column="PEST", legend=True, scheme="User_Defined", cmap="YlOrBr", 
         edgecolor="grey", classification_kwds=dict(bins=[40,60,80,100,120]),
         legend_kwds={"labels": ["< 40", "40 - 60", "60 - 80", "80 - 100", "100 - 120", "> 120"]})
mnp["coords"] = mnp["geometry"].apply(lambda x: x.representative_point().coords[:])
mnp["coords"] = [coords[0] for coords in mnp["coords"]]
for idx, row in mnp.iterrows():
    ax.annotate(text=idx, xy=row["coords"],
                 horizontalalignment="center")
ax.set_title("Minnesota Pest Pressure for selected counties", weight="bold")

### Spatial weights (contiguity and distance-based)

#### Queen's case (contiguity)

In [ ]:
# calculate neighboring using Queen's case (contiguity)
mnp_nbq = weights.contiguity.Queen.from_dataframe(mnp)
# summarize 
print ("Number of units: {}".format(mnp_nbq.n))
print ("Number of nonzero weights: {}".format(mnp_nbq.nonzero))
print ("Percentage of nonzero weights: {}".format(mnp_nbq.pct_nonzero))
print ("Average number of neighbors: {}".format(mnp_nbq.mean_neighbors))
print ("Largest number of neighbors is {}".format(mnp_nbq.max_neighbors))
print ("Minimum number of neighbors is {}".format(mnp_nbq.min_neighbors))
print ("Number of units without any neighbors {}".format(len(mnp_nbq.islands)))
print ("Histogram: {}".format(mnp_nbq.histogram))
print ("Neighbour list: {}".format(mnp_nbq.neighbors))

#### Rook's case (contiguity)

In [ ]:
# calculate neighboring using Rook's case (contiguity)
mnp_nbr = weights.contiguity.Rook.from_dataframe(mnp)
# summarize 
print ("Number of units: {}".format(mnp_nbr.n))
print ("Number of nonzero weights: {}".format(mnp_nbr.nonzero))
print ("Percentage of nonzero weights: {}".format(mnp_nbr.pct_nonzero))
print ("Average number of neighbors: {}".format(mnp_nbr.mean_neighbors))
print ("Largest number of neighbors is {}".format(mnp_nbr.max_neighbors))
print ("Minimum number of neighbors is {}".format(mnp_nbr.min_neighbors))
print ("Number of units without any neighbors {}".format(len(mnp_nbr.islands)))
print ("Histogram: {}".format(mnp_nbr.histogram))
print ("Neighbour list: {}".format(mnp_nbr.neighbors))

#### K-nearest neighbors (distance-based)

In [ ]:
# calculate neighboring using K-nearest neighbors (distance-based)
mnp_nbk3 = weights.distance.KNN.from_dataframe(mnp, k=3)
# summarize 
print ("Number of units: {}".format(mnp_nbk3.n))
print ("Number of nonzero weights: {}".format(mnp_nbk3.nonzero))
print ("Percentage of nonzero weights: {}".format(mnp_nbk3.pct_nonzero))
print ("Average number of neighbors: {}".format(mnp_nbk3.mean_neighbors))
print ("Largest number of neighbors is {}".format(mnp_nbk3.max_neighbors))
print ("Minimum number of neighbors is {}".format(mnp_nbk3.min_neighbors))
print ("Number of units without any neighbors {}".format(len(mnp_nbk3.islands)))
print ("Histogram: {}".format(mnp_nbk3.histogram))
print ("Neighbour list: {}".format(mnp_nbk3.neighbors))

#### Distance (distance-based)

In [ ]:
# calculate neighboring by distance (distance-based)
mnp_nbd = weights.distance.DistanceBand.from_dataframe(mnp, 100000, binary=True)
# summarize 
print ("Number of units: {}".format(mnp_nbd.n))
print ("Number of nonzero weights: {}".format(mnp_nbd.nonzero))
print ("Percentage of nonzero weights: {}".format(mnp_nbd.pct_nonzero))
print ("Average number of neighbors: {}".format(mnp_nbd.mean_neighbors))
print ("Largest number of neighbors is {}".format(mnp_nbd.max_neighbors))
print ("Minimum number of neighbors is {}".format(mnp_nbd.min_neighbors))
print ("Number of units without any neighbors {}".format(len(mnp_nbd.islands)))
print ("Histogram: {}".format(mnp_nbd.histogram))
print ("Neighbour list: {}".format(mnp_nbd.neighbors))

### Visualize and compare weights networks

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 9))
plot_spatial_weights(mnp_nbq, mnp, ax=axs[0, 0])
axs[0, 0].set_title("Queen's Case Contiguity")
plot_spatial_weights(mnp_nbr, mnp, ax=axs[0, 1])
axs[0, 1].set_title("Rook's Case Contiguity")
plot_spatial_weights(mnp_nbk3, mnp, ax=axs[1, 0])
axs[1, 0].set_title("K-nearest Neighbors (k=3)")
plot_spatial_weights(mnp_nbd, mnp, ax=axs[1, 1])
axs[1, 1].set_title("Distance (100,000)")
plt.tight_layout()

### Spatial weights transformation and weights summary
In this examples, we are setting transformations of weights and then computing an adjacency list representation of a weights object. Two different transforms are presented: `R` – Row-standardization, and `B` – Binary.

In [ ]:
# Spatial Weights Summary - Row Standardized
mnp_nbq.set_transform("R")
mnp_nbq_lw_r = mnp_nbq.to_adjlist()
print ("Property s0 = {}".format(mnp_nbq.s0))
print ("Property s1 = {}".format(mnp_nbq.s1))
print ("Property s2 = {}".format(mnp_nbq.s2))
print ("Weights: {}".format(mnp_nbq.weights))
print ("Weights summary: ")
print (mnp_nbq_lw_r["weight"].describe())

In [ ]:
# Spatial Weights Summary - Binary
mnp_nbq.set_transform("B")
mnp_nbq_lw_b = mnp_nbq.to_adjlist()
print ("Property s0 = {}".format(mnp_nbq.s0))
print ("Property s1 = {}".format(mnp_nbq.s1))
print ("Property s2 = {}".format(mnp_nbq.s2))
print ("Weights: {}".format(mnp_nbq.weights))
print ("Weights summary: ")
print (mnp_nbq_lw_b["weight"].describe())

### Spatial Autocorrelation with Moran’s I Global Statistic
Moran's I statistic measures spatial autocorrelation based on feature locations and feature values simultaneously. It allows to evaluate whether the pattern presented by the features is clustered, dispersed, or random.

In [ ]:
mi = esda.moran.Moran(mnp["PEST"], mnp_nbq)
print ("Moran's I statistic: {}".format(mi.I))
print ("p-value of I under randomization assumption: {}".format(mi.p_rand))
print ("variance of I under randomization assumption: {}".format(mi.VI_rand))
print ("Expected value under normality assumption: {}".format(mi.EI))

### Visualize Moran's I plot

In [ ]:
plot_moran(mi, zstandard=False, figsize=(10,4))
plt.show()

### Spatial Regression Models

#### ML estimation of the spatial lag model

In [ ]:
y = mnp["PEST"].to_numpy()
x = mnp[["HOST"]].values
mnp_slm = ML_Lag(y, x, mnp_nbq, name_w="Queen's Case", name_x=["HOST"], name_y="PEST", 
                 name_ds="MN Pest Pressure")
print ("Estimate of spatial autoregressive coefficient rho: {}".format(mnp_slm.rho))
print(mnp_slm.summary)

#### ML estimation of the spatial error model

In [ ]:
mnp_sem = ML_Error(y, x, mnp_nbq, name_w="Queen's Case", name_x=["HOST"], name_y="PEST", 
                   name_ds="MN Pest Pressure")
print(mnp_sem.summary)

#### Spatial Durbin model
Although some models are not directly offered by PySal APIs, they can be derived from existing standard models. For example, a spatial Durbin model can be estimated by computing a spatial lag of  independent variables and then adding the set of lagged variables to the original independent variables to run a spatial lag model.  

In [ ]:
lag_x = weights.lag_spatial(mnp_nbq, x)
new_x = np.hstack((x,lag_x))
mnp_sdm = ML_Lag(y, new_x, mnp_nbq, name_w="Queen's Case", name_x=["HOST"], name_y="PEST", 
                 name_ds="MN Pest Pressure")
print ("Estimate of spatial autoregressive coefficient rho: {}".format(mnp_sdm.rho))
print(mnp_sdm.summary)

#### Ordinary least squares model

In [ ]:
mnp_ols = OLS(y, x, mnp_nbq, name_w="Queen's Case", name_x=["HOST"], name_y="PEST", 
                 name_ds="MN Pest Pressure", spat_diag=True)
print(mnp_ols.summary)

#### Moran's I test on ordinary least squares model residuals

In [ ]:
mi_ols = esda.moran.Moran(mnp_ols.u, mnp_nbq)
print ("Moran's I statistic: {}".format(mi_ols.I))
print ("p-value of I under randomization assumption: {}".format(mi_ols.p_rand))
print ("variance of I under randomization assumption: {}".format(mi_ols.VI_rand))
print ("Expected value under normality assumption: {}".format(mi_ols.EI))

In [ ]:
plot_moran(mi_ols, zstandard=False, figsize=(10,4))
plt.show()

## Exercises
For the exercies, data are created and saved to the directory `./data-module-4/`.
- `mwi.shp` -  a dataset downloaded from the Malawi Living Standard Measurement Survey Integrated Household Sample (LSMS-IHS) Data Wave 5 Data (available from https://microdata.worldbank.org/index.php/catalog/3818).

**Question 1. Read the vector dataset `mwi.shp` into a `GeoDataFrame`. Print its Coordinate Reference System.**

**Question 2. Calculate neighboring using Queen's case (contiguity), Rook's case (contiguity), K-nearest neighbors (k=3), and distance (200,000 m). Print the properties for each neighbouring, such as number of units, number of nonzero weights, etc.**

**Question 3. Visualize and compare all 4 weights networks. What differences do you see?**

**Question 4. Apply row-standardized and binary transforms to your Queen's case neighbourhood. Compare their summaries.**

**Question 5. Run the Moran's I statistic to test the spatial autocorrelation for `poverty` variable. Use Queen's case neighbouring structure.**

**Question 6. Visualize Moran's I plot for `poverty` variable.**

**Question 7. Run ML estimation of the spatial lag model. Use Queen's case neighbouring structure. Predict `poverty` as a function of cropland cultivated (`croplnd`), livestock owned (`livstck`), share of off-farm income (`income`), years of education (`edu`), female head of household (`female`) and tobacco growing household (`tobccHH`).**

**Question 8. Run the Ordinary least squares model. Use Queen's case neighbouring structure. Use the same `x` and `y` variables as for the previous question.**